
# Operational-Space Servoing in MuJoCo (Franka Panda)  
**Sine/Cosine ellipse and horizontal figure-8 trajectories** with live visualization of both the **target** and the **actual end-effector**.

This notebook will:
1. **Install** required packages (`mujoco>=3.1.0`, `glfw`, `numpy`).  
2. **Download** the Franka Panda model from DeepMind's *MuJoCo Menagerie* (the `franka_emika_panda/` folder).  
3. Provide a **gentle mathematical background**: task-space error → twist, Jacobian, **Damped Least Squares** (DLS) IK, and **nullspace** posture control.  
4. Run a **real-time controller** that tracks either an **ellipse** or a **horizontal figure‑8** trajectory in the end-effector’s **XY plane**.  
5. Visualize both the **target path (blue)** and the **actual path (green)** inside MuJoCo’s viewer by drawing tiny spheres in the user scene.



## 1) Install requirements
Run the cell below. If packages are already installed, pip will skip or upgrade them.

> **Notes**
> - You need a machine with an OpenGL-capable environment to open the interactive MuJoCo viewer window.  
> - On some remote/WSL setups you might need: `export MUJOCO_GL=egl` (or `osmesa`) before launching Python. See the *Troubleshooting* section at the end.


In [1]:
# If you are on a clean environment, uncomment the next line or just run it as-is.
# The exact MuJoCo minor version can be adjusted if needed; 3.1.5 is a common stable pick.
!python -m pip install -U mujoco glfw numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 2.6 MB/s eta 0:00:002.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 21.1 MB/s eta 0:00:0031m23.8 MB/s eta 0:00:01
  Attempting uninstall: glfw
    Found existing installation: glfw 2.9.0
    Uninstalling glfw-2.9.0:
      Successfully uninstalled glfw-2.9.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
  Attempting uninstall: mujoco━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [numpy]
    Found existing installation: mujoco 3.3.4╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [numpy]
    Uninstalling mujoco-3.3.4:╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [numpy]
      Successfully uninstalled mujoco-3.3.47m╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [numpy]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [mujoco]━━━━━━━━━━━ 2/3 [mujoco]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip



## 2) Download the Franka model folder
If you already have `franka_emika_panda/` models, you can just skip this.

We fetch the `franka_emika_panda/` directory from DeepMind's **MuJoCo Menagerie**.  
The cell tries **Option A (git)** first; if `git` is unavailable, it falls back to **Option B (zip)**.

After this cell, the local path `./franka_emika_panda/scene.xml` should exist.


In [14]:

# !!! If you already have `franka_emika_panda/`, you can just skip this.
import os, shutil, zipfile, sys

# Try Option A: git clone (shallow)
try:
    if not os.path.isdir("mujoco_menagerie"):
        !git clone --depth=1 https://github.com/google-deepmind/mujoco_menagerie.git
    # Copy Franka folder next to this notebook for stable relative paths
    if os.path.isdir("mujoco_menagerie/franka_emika_panda"):
        if os.path.isdir("franka_emika_panda"):
            shutil.rmtree("franka_emika_panda")
        shutil.copytree("mujoco_menagerie/franka_emika_panda", "franka_emika_panda")
    else:
        raise RuntimeError("Franka folder not found inside mujoco_menagerie.")
except Exception as e:
    print("[git failed or unavailable] Falling back to ZIP method:", e)
    # Option B: download ZIP and extract only the Franka folder
    import urllib.request, tempfile
    url = "https://github.com/google-deepmind/mujoco_menagerie/archive/refs/heads/main.zip"
    with tempfile.TemporaryDirectory() as td:
        zip_path = os.path.join(td, "menagerie.zip")
        print("Downloading:", url)
        urllib.request.urlretrieve(url, zip_path)
        with zipfile.ZipFile(zip_path) as zf:
            prefix = "mujoco_menagerie-main/franka_emika_panda/"
            members = [m for m in zf.namelist() if m.startswith(prefix)]
            if not members:
                raise RuntimeError("Could not locate Franka folder in ZIP.")
            zf.extractall(td, members)
        src = os.path.join(td, "mujoco_menagerie-main", "franka_emika_panda")
        if os.path.isdir("franka_emika_panda"):
            shutil.rmtree("franka_emika_panda")
        shutil.copytree(src, "franka_emika_panda")

# Verify result
print("Contents of ./franka_emika_panda:")
print(os.listdir("franka_emika_panda"))
assert os.path.exists("franka_emika_panda/scene.xml"), "scene.xml not found."


Cloning into 'mujoco_menagerie'...
remote: Enumerating objects: 2097, done.
remote: Counting objects: 100% (2097/2097), done.
remote: Compressing objects: 100% (1905/1905), done.
remote: Total 2097 (delta 225), reused 1780 (delta 185), pack-reused 0 (from 0)
Receiving objects: 100% (2097/2097), 359.64 MiB | 15.68 MiB/s, done.
Resolving deltas: 100% (225/225), done.
Updating files: 100% (2109/2109), done.
Contents of ./franka_emika_panda:
['mjx_panda_nohand.xml', 'mjx_single_cube.xml', 'assets', 'hand.xml', 'panda_nohand.xml', 'mjx_scene.xml', 'mjx_panda.xml', 'CHANGELOG.md', 'scene.xml', 'LICENSE', 'panda.xml', 'README.md', 'panda.png']



## 3) Mathematical background (friendly version)

We want the robot **end‑effector** (EE) to follow a desired trajectory in **task space** (the world coordinate frame).  
At each time step we do four things:

### (a) Pose error → “twist” (desired spatial velocity)
- Position error:  $\mathbf{e}_p = \mathbf{p}^* - \mathbf{p}$.
- Orientation error: use unit quaternions. If $\mathbf{q}$ is the current EE orientation and $\mathbf{q}^*$ is the target, the **error quaternion** is  
  $\mathbf{q}_\text{err} = \mathbf{q}^* \otimes \mathbf{q}^{-1}$.
- MuJoCo provides a handy routine to convert this quaternion error to an equivalent **angular velocity** $\boldsymbol{\omega}_d$ (via `mju_quat2Vel`).

We then build the 6‑D **twist** (linear + angular velocities):  
$\mathbf{v}_d = \begin{bmatrix} \mathbf{v}_d^\text{lin} \\ \boldsymbol{\omega}_d \end{bmatrix}$,  
with simple proportional terms:  
$\mathbf{v}_d^\text{lin} = K_\text{pos}\, \mathbf{e}_p / \Delta t_\text{int}$,  
$\boldsymbol{\omega}_d = K_\text{ori}\, \boldsymbol{\omega}_d / \Delta t_\text{int}$.

Intuition: if $K \approx 1$, the controller tries to “close the gap” within one integration step $\Delta t_\text{int}$.

### (b) Map twist to joint velocities using the Jacobian
Let $\mathbf{J} \in \mathbb{R}^{6 \times n}$ be the EE geometric Jacobian at the current configuration. We want  
$\mathbf{J}\,\dot{\mathbf{q}} \approx \mathbf{v}_d$.

### (c) Damped Least Squares (DLS)
To avoid huge joint velocities near kinematic singularities we use **DLS**:  
$\displaystyle \dot{\mathbf{q}} = \mathbf{J}^\top\,(\mathbf{J}\mathbf{J}^\top + \lambda^2 \mathbf{I}_6)^{-1}\,\mathbf{v}_d$.

- $\lambda > 0$ is the **damping** (small but non‑zero).  
- If $\lambda \to 0$, this approaches the minimum‑norm least‑squares solution.  
- If $\lambda$ is larger, the solution becomes more conservative (safer near singularities).

### (d) Nullspace posture control
We can bias the robot posture toward a comfortable “home” configuration $\mathbf{q}_0$ **without** affecting the task:  
$\displaystyle \dot{\mathbf{q}}_\text{total} = \dot{\mathbf{q}}_\text{DLS} + (\mathbf{I} - \mathbf{J}^+ \mathbf{J})\,K_n\,(\mathbf{q}_0 - \mathbf{q})$,  
where $\mathbf{J}^+$ is the (Moore–Penrose) pseudoinverse, and $K_n$ is a diagonal gain. The projector $(\mathbf{I} - \mathbf{J}^+\mathbf{J})$ keeps this term in the **nullspace** of the task, so it does **not** change the end‑effector motion.

### (e) Integrate and respect joint limits
We integrate joint velocities for $\Delta t_\text{int}$ to get a new reference position $\mathbf{q}_\text{ref}$, clip it to joint limits, and send it to the actuators as a **position command**.

> In words: every small step, we point the end‑effector toward the target, map that to joint speeds in a safe way, keep the posture nice, and take a small step.



## 4) Helper: draw tiny spheres in the MuJoCo viewer (user scene)
`mujoco.viewer` does **not** have `add_marker`. Instead we add `mjvGeom` objects into the **user scene** (`viewer.user_scn`).  
The function below takes care of data types and shapes expected by `mjv_initGeom`.


In [2]:

import numpy as np
import mujoco

def add_sphere_to_user_scene(scn: mujoco.MjvScene, pos, radius, rgba):
    """Add one small sphere into the viewer's user scene.
    - scn: viewer.user_scn (MjvScene)
    - pos: (3,) array-like, world position of the sphere center
    - radius: float, sphere radius in meters
    - rgba: (4,) array-like in [0,1], float32
    """
    if scn.ngeom >= scn.maxgeom:
        return  # Avoid overflow

    size = np.array([radius, 0.0, 0.0], dtype=np.float64)   # len=3, float64
    pos64 = np.asarray(pos, dtype=np.float64)               # len=3, float64
    mat9 = np.eye(3, dtype=np.float64).reshape(-1)          # len=9, float64
    rgba32 = np.asarray(rgba, dtype=np.float32)             # len=4, float32

    g = scn.geoms[scn.ngeom]
    mujoco.mjv_initGeom(
        g,
        mujoco.mjtGeom.mjGEOM_SPHERE,
        size,
        pos64,
        mat9,
        rgba32,
    )
    scn.ngeom += 1



## 5) Configuration and constants
- **Trajectory mode**: `"ellipse"` or `"figure8"` (horizontal **8** on the XY plane using the Gerono curve).  
- Gains and timing: `Kpos`, `Kori`, `integration_dt`, and the simulation time step `dt`.  
- Nullspace gains `Kn` and a joint velocity limit for safety.


In [11]:

from collections import deque

# ---- Control parameters ----
integration_dt: float = 0.1     # "desired one-step-to-go" horizon (s)
damping: float = 1e-4           # DLS damping (lambda^2 in derivation)
Kpos: float = 0.95              # 0..1 proportional gain on position
Kori: float = 0.95              # 0..1 proportional gain on orientation
gravity_compensation: bool = True
dt: float = 0.002               # simulation step (s)
Kn = np.asarray([10.0, 10.0, 10.0, 10.0, 5.0, 5.0, 5.0])  # nullspace P
max_angvel = 0.785              # rad/s max |dq_i|

# ---- Trajectory parameters ----
use_scripted_traj = True        # True: scripted target; False: drag with mouse
traj_mode = "figure8"           # "ellipse" or "figure8"
Ax, Ay, Az = 0.1, 0.2, 0.00   # amplitudes (m); Az=0 => XY plane
freq = 0.25                     # Hz (increase for faster motion)
marker_interval = 0.05          # seconds between trajectory dots

# Optional: for some remote/WSL setups, uncomment one of these:
# import os; os.environ.setdefault("MUJOCO_GL", "egl")
# import os; os.environ.setdefault("MUJOCO_GL", "osmesa")



## 6) Main control loop
This is the end-effector servoing controller you asked for, adapted for notebook use and with detailed comments:

**What it does each step**
1. Update a **scripted target** pose (`ellipse` or `figure8`) in the **mocap body** named `target`.  
2. Compute the **twist** from pose error (position + quaternion orientation).  
3. Solve **DLS** for joint velocities; add the **nullspace** bias toward the `home` keyframe.  
4. Integrate velocities into a new joint reference position and send it to actuators.  
5. Draw **blue dots** for the target path and **green dots** for the actual EE path.


In [12]:

import time
import mujoco
import numpy as np

def run_viewer(traj_mode: str = "figure8"):
    assert mujoco.__version__ >= "3.1.0", "Please use mujoco 3.1.0 or later."

    # Load model and data
    model = mujoco.MjModel.from_xml_path("franka_emika_panda/scene.xml")
    data = mujoco.MjData(model)

    # Basic sim options
    model.body_gravcomp[:] = float(gravity_compensation)
    model.opt.timestep = dt

    # End-effector site and IDs
    site_name = "attachment_site"
    site_id = model.site(site_name).id
    joint_names = ["joint1","joint2","joint3","joint4","joint5","joint6","joint7"]
    dof_ids = np.array([model.joint(name).id for name in joint_names])
    actuator_ids = np.array([model.actuator(name).id for name in joint_names])

    # Keyframe: "home" pose for posture bias
    key_name = "home"
    key_id = model.key(key_name).id
    q0 = model.key(key_name).qpos

    # Mocap target
    mocap_name = "target"
    mocap_id = model.body(mocap_name).mocapid[0]

    # Preallocations
    jac = np.zeros((6, model.nv))      # EE Jacobian (6 x nv)
    diag = damping * np.eye(6)         # DLS damping matrix
    eye = np.eye(model.nv)             # identity (nv x nv)
    twist = np.zeros(6)                # desired spatial velocity
    site_quat = np.zeros(4)
    site_quat_conj = np.zeros(4)
    error_quat = np.zeros(4)

    # Launch viewer
    with mujoco.viewer.launch_passive(
        model=model,
        data=data,
        show_left_ui=False,
        show_right_ui=False,
    ) as viewer:

        # Reset to keyframe and set up camera
        mujoco.mj_resetDataKeyframe(model, data, key_id)
        mujoco.mjv_defaultFreeCamera(model, viewer.cam)
        viewer.opt.frame = mujoco.mjtFrame.mjFRAME_SITE  # show site frame

        # Use the current EE pose as the trajectory center and reference orientation
        mujoco.mj_forward(model, data)
        p_center = data.site(site_id).xpos.copy()
        site_quat_home = np.zeros(4)
        mujoco.mju_mat2Quat(site_quat_home, data.site(site_id).xmat)

        # Initialize mocap target at the center (avoid big jump)
        data.mocap_pos[mocap_id] = p_center
        data.mocap_quat[mocap_id] = site_quat_home

        # Trajectory point buffers (limit to half of user_scn capacity)
        from collections import deque
        max_points = max(200, min(2000, viewer.user_scn.maxgeom // 2))
        tgt_pts = deque(maxlen=max_points)
        ee_pts  = deque(maxlen=max_points)

        t0 = time.time()
        last_mark = t0

        while viewer.is_running():
            step_start = time.time()
            t = step_start - t0
            omega = 2 * np.pi * freq

            # 1) Scripted target (ellipse or figure-8 in XY plane)
            if use_scripted_traj:
                if traj_mode == "ellipse":
                    p_tgt = p_center + np.array([
                        Ax * np.sin(omega * t),
                        Ay * np.cos(omega * t),
                        0.0 + Az * 0.0
                    ])
                elif traj_mode == "figure8":
                    # Gerono figure-8 (two loops stacked along Y):
                    # y = Ay * sin(ωt),  x = (Ax/2) * sin(2ωt)
                    p_tgt = p_center + np.array([
                        0.5 * Ax * np.sin(2 * omega * t),
                        Ay * np.sin(omega * t),
                        0.0
                    ])
                else:
                    raise ValueError(f"Unknown traj_mode: {traj_mode}")

                q_tgt = site_quat_home  # keep orientation fixed (can be animated too)
                data.mocap_pos[mocap_id] = p_tgt
                data.mocap_quat[mocap_id] = q_tgt

            # 2) Task-space error -> twist
            dx = data.mocap_pos[mocap_id] - data.site(site_id).xpos
            twist[:3] = Kpos * dx / integration_dt
            mujoco.mju_mat2Quat(site_quat, data.site(site_id).xmat)
            mujoco.mju_negQuat(site_quat_conj, site_quat)
            mujoco.mju_mulQuat(error_quat, data.mocap_quat[mocap_id], site_quat_conj)
            mujoco.mju_quat2Vel(twist[3:], error_quat, 1.0)  # angular vel from quat err
            twist[3:] *= Kori / integration_dt

            # 3) Jacobian + DLS
            mujoco.mj_jacSite(model, data, jac[:3], jac[3:], site_id)
            dq = jac.T @ np.linalg.solve(jac @ jac.T + diag, twist)

            # 4) Nullspace bias toward 'home'
            dq += (eye - np.linalg.pinv(jac) @ jac) @ (Kn * (q0 - data.qpos[dof_ids]))

            # 5) Clamp, integrate, and send as position control
            dq_abs_max = np.abs(dq).max()
            if dq_abs_max > max_angvel:
                dq *= max_angvel / dq_abs_max

            q = data.qpos.copy()
            mujoco.mj_integratePos(model, q, dq, integration_dt)
            np.clip(q, *model.jnt_range.T, out=q)
            data.ctrl[actuator_ids] = q[dof_ids]

            # 6) Visualization: dots for target (blue) and actual EE (green)
            now = step_start
            if now - last_mark >= marker_interval:
                tgt_pts.append(data.mocap_pos[mocap_id].copy())
                ee_pts.append(data.site(site_id).xpos.copy())
                last_mark = now
                with viewer.lock():
                    scn = viewer.user_scn
                    scn.ngeom = 0  # rebuild only our custom dots
                    for p in tgt_pts:
                        add_sphere_to_user_scene(scn, p, radius=0.005, rgba=[1.0, 0.0, 0.0, 0.8])
                    for p in ee_pts:
                        add_sphere_to_user_scene(scn, p, radius=0.005, rgba=[0.2, 1.0, 0.2, 0.8])

            # 7) Sim step and sync
            mujoco.mj_step(model, data)
            viewer.sync()

            # Keep real-time pace
            sleep_time = dt - (time.time() - step_start)
            if sleep_time > 0:
                time.sleep(sleep_time)



## 7) Run it!
- Set `traj_mode` to either `"ellipse"` or `"figure8"` in the configuration cell.  
- Then run the cell below. A window should pop up and the end-effector will track the target path.  
- **Blue** dots = target mocap path. **Green** dots = actual end‑effector path.

> Close the window to stop the loop.


In [13]:
import mujoco.viewer
# Choose which trajectory to run ("ellipse" or "figure8")
run_viewer(traj_mode=traj_mode)



## 8) Troubleshooting

- **`AttributeError: 'Handle' object has no attribute 'add_marker'`**  
  The modern `mujoco.viewer` does not have `add_marker`. This notebook draws dots by adding `mjvGeom` items into `viewer.user_scn`.

- **`Xlib: extension "NV-GLX" missing on display`** or **viewer window fails to open**  
  Your environment may lack an OpenGL display (e.g., remote or WSL). Try:
  ```bash
  export MUJOCO_GL=egl   # or: export MUJOCO_GL=osmesa
  ```
  Set it *before* launching Python/Jupyter. On Windows/macOS with a normal desktop, you should not need this.

- **The robot does not exactly follow the path**  
  Increase `Kpos`/`Kori` slightly (≤ 1.0), or reduce `freq`/amplitudes. Near singularities, a higher `damping` helps.

- **Too many dots**  
  Increase `marker_interval` or reduce the deque length inside the code (it auto-limits to half of `user_scn.maxgeom`).

- **Change the figure‑8 shape**  
  The current figure‑8 is the *Gerono* curve:  
  $x(t)=0.5 A_x \sin(2 \omega t),\; y(t)=A_y \sin(\omega t)$.  
  Swap x/y if you want a sideways ∞‑shape.
